In [1]:
# Library
import torch
import numpy as np
import matplotlib as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
# device 설정
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')

print(DEVICE)

cuda


In [3]:
# Hyperparameter setting
batch_size = 32
epochs = 10

In [4]:
# MNIST DATA LOAD
train_dataset = datasets.MNIST(root="../data/MNIST", train = True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST(root="../data/MNIST", train = False, transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size=batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size=batch_size, shuffle = True)

In [5]:
# MLP 모듈 설계 + Dropout

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(28*28 , 512)
        self.fc2 = nn.Linear(512 , 256)
        self.fc3 = nn.Linear(256 , 10)
        self.dropout_prob = 0.5
    
    def forward(self, x):
        x = x.view(-1, 28*28) # flatten 느낌
        x = self.fc1(x)
        x = F.sigmoid(x)
        x = F.dropout(x, training=self.training, p = self.dropout_prob)
        x = self.fc2(x)
        x = F.sigmoid(x)
        x = F.dropout(x, training=self.training, p = self.dropout_prob)
        x = self.fc3(x)
        x = F.log_softmax(x, dim=1)
        return x

In [6]:
# optimizer, obj function 설정
model = Net().to(DEVICE)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01, momentum=0.5)
criterian = nn.CrossEntropyLoss()
print(model)

Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
)


In [7]:
# 학습 데이터에 대한 model 성능 확인 함수 정의
def train(model, train_loader, optimizer, log_interval):
    model.train()
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        output = model(image)
        loss = criterian(output,label)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print("Train_epoch : {} [{}/{}({:.0f}%)] \tTrain Loss : {:.6f}".format(epochs, batch_idx*len(image),len(train_loader.dataset), 100*batch_idx/len(train_loader), loss.item()))

In [8]:
# 검증 데이터에 대한 모델 성능 확인하는 함수 정의
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += criterian(output,label).item()
            prediction = output.max(1, keepdim = True)[1]
            correct +=prediction.eq(label.view_as(prediction)).sum().item() 

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100 * correct / len(test_loader.dataset)
    return test_loss, test_accuracy


In [9]:
# Train, Test 진행
for epoch in range(1, epochs+1):
    train(model, train_loader, optimizer, log_interval = 200)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("\n[Epoch: {}], \tTest loss: {:.4f}, \tTest acc : {:.2f}".format(epoch, test_loss, test_accuracy))

/home/seonggye/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train_epoch : 10 [0/60000(0%)] 	Train Loss : 2.433305
Train_epoch : 10 [6400/60000(11%)] 	Train Loss : 2.369272
Train_epoch : 10 [12800/60000(21%)] 	Train Loss : 2.375231
Train_epoch : 10 [19200/60000(32%)] 	Train Loss : 2.434787
Train_epoch : 10 [25600/60000(43%)] 	Train Loss : 2.324700
Train_epoch : 10 [32000/60000(53%)] 	Train Loss : 2.314241
Train_epoch : 10 [38400/60000(64%)] 	Train Loss : 2.196862
Train_epoch : 10 [44800/60000(75%)] 	Train Loss : 2.297035
Train_epoch : 10 [51200/60000(85%)] 	Train Loss : 2.359604
Train_epoch : 10 [57600/60000(96%)] 	Train Loss : 2.265689

[Epoch: 1], 	Test loss: 0.0712, 	Test acc : 20.01
Train_epoch : 10 [0/60000(0%)] 	Train Loss : 2.344600
Train_epoch : 10 [6400/60000(11%)] 	Train Loss : 2.311766
Train_epoch : 10 [12800/60000(21%)] 	Train Loss : 2.251694
Train_epoch : 10 [19200/60000(32%)] 	Train Loss : 2.308923
Train_epoch : 10 [25600/60000(43%)] 	Train Loss : 2.231268
Train_epoch : 10 [32000/60000(53%)] 	Train Loss : 2.208254
Train_epoch : 10 

In [ ]:
# MLP 모듈 설계 + Dropout + batch normalizeation

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(28*28 , 512)
        self.fc2 = nn.Linear(512 , 256)
        self.fc3 = nn.Linear(256 , 10)
        self.dropout_prob = 0.5
        self.batch_norm1 = nn.BatchNorm1d(512)
        self.batch_norm1 = nn.BatchNorm1d(256)
    
    def forward(self, x):
        x = x.view(-1, 28*28) # flatten 느낌
        x = self.fc1(x)
        x = self.batch_norm1(x)
        x = F.sigmoid(x)
        x = F.dropout(x, training=self.training, p = self.dropout_prob)
        x = self.fc2(x)
        x = self.batch_norm2(x)
        x = F.sigmoid(x)
        x = F.dropout(x, training=self.training, p = self.dropout_prob)
        x = self.fc3(x)
        x = F.log_softmax(x, dim=1)
        return x

In [10]:
# weight initialization
import torch.nn.init as init 
def weight_init(m):
    if isinstance(m, nn.Linear):
        init.kaiming_uniform_(m.weight.data)

model = Net().to(DEVICE)
model.apply(weight_init)

Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
)